In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# 데이터 읽어오기
train_file = "./models/total_train_data.csv"
data = pd.read_csv(train_file, delimiter=',')
queries = data['query'].tolist()
intents = data['intent'].tolist()

from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic='./tools/dict/chatbot_dict.bin',
               userdic='./utils/user_dic.tsv')

# 단어 시퀀스 생성
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기

In [2]:
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [3]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [4]:
# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

# CNN 모델 정의  ○4
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

In [5]:
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(5, name='logits')(dropout_hidden)
predictions = Dense(5, activation=tf.nn.softmax)(logits)

In [6]:
#모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)


# 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))


# 모델 저장
model.save('intent_model.h5')

Epoch 1/5
3698/3698 [==============================] - 55s 12ms/step - loss: 0.0486 - accuracy: 0.9845 - val_loss: 0.0136 - val_accuracy: 0.9961
Epoch 2/5
3698/3698 [==============================] - 40s 11ms/step - loss: 0.0150 - accuracy: 0.9948 - val_loss: 0.0085 - val_accuracy: 0.9963
Epoch 3/5
3698/3698 [==============================] - 37s 10ms/step - loss: 0.0100 - accuracy: 0.9961 - val_loss: 0.0082 - val_accuracy: 0.9967
Epoch 4/5
3698/3698 [==============================] - 37s 10ms/step - loss: 0.0101 - accuracy: 0.9962 - val_loss: 0.0068 - val_accuracy: 0.9971
Epoch 5/5
529/529 [==============================] - 2s 4ms/step - loss: 0.0056 - accuracy: 0.9973
Accuracy: 99.734974
loss: 0.005582


### Test

In [12]:
from utils.Preprocess import Preprocess
from models.intent.intentModel import IntentModel

p = Preprocess(word2index_dic='./tools/dict/chatbot_dict.bin',
               userdic='../utils/user_dic.tsv')

intent = IntentModel(model_name='./models/intent/intent_model.h5', proprocess=p)

In [32]:
query = "탕수육 얼마에요?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 21ms/step
탕수육 얼마에요?
의도 예측 클래스 :  2
의도 예측 레이블 :  주문


## 개체명 인식

In [ ]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))

    return sents

# 전처리 객체 생성
p = Preprocess(word2index_dic='./tools/dict/chatbot_dict.bin',
                  userdic = './utils/user_dic.tsv')

# 학습용 말뭉치 데이터 불러옴
corpus = read_file('./models/ner/ner_train.txt')